<a href="https://colab.research.google.com/github/maddheshiya937/Event-Manager/blob/main/predicting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import re
from nltk import word_tokenize, FreqDist
import nltk
nltk.download('punkt')
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
import hashlib
import time
import binascii
import string
import os, sys
import pickle
import os
from keras.models import load_model
from keras.utils import pad_sequences

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install pefile
!pip install yara-python
!apt install python-magic

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.8/71.8 kB 8.4 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 538.5/538.5 kB 31.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for yara-python: filename=yara_python-4.3.1-cp310-cp310-linux_x86_64.whl size=865517 sha256=fdc26ed35cc64715d817f10a88bdca98fbab8d394c9d96deea8e1d29cb0ab835
  Stored in directory: /root/.cache/pip/wheels/e2/3c/18/482952dad97296f5feeea6e93d5bf8fb9c695770314c0478d7
Successfully built yara-python
Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  libpython2-stdlib python2 python2-minimal
Suggested packages:
  python2-doc python-tk
The following NEW packages will be installed:
  libp

In [ ]:
# import commands

try:
    import pefile
    import peutils
except ImportError:
    print ('pefile not installed, see http://code.google.com/p/pefile/')
    sys.exit()

try:
    import magic
except ImportError:
    print ('python-magic is not installed, file types will not be available')

try:
    import yara
except ImportError:
    print ('yara-python is not installed, see http://code.google.com/p/yara-project/')

# suspicious APIs to alert on
alerts = ['OpenProcess', 'VirtualAllocEx', 'WriteProcessMemory', 'CreateRemoteThread', 'ReadProcessMemory',
          'CreateProcess', 'WinExec', 'ShellExecute', 'HttpSendRequest', 'InternetReadFile', 'InternetConnect',
          'CreateService', 'StartService']

# legit entry point sections
good_ep_sections = ['.text', '.code', 'CODE', 'INIT', 'PAGE']

# path to clamscan (optional)
clamscan_path = ('/usr/bin/clamscanx')

def header(msg):
    return "\n" + msg + "\n" + ("=" * 90)
    
def convert_char(char):
    if char in string.ascii_letters or \
       char in string.digits or \
       char in string.punctuation or \
       char in string.whitespace:
        return char
    else:
        return r'\x%02x' % ord(char)

def convert_to_printable(s):
    return ''.join([convert_char(c) for c in s])

def get_filetype(data):
    """There are two versions of python-magic floating around, and annoyingly, the interface
    changed between versions, so we try one method and if it fails, then we try the other.
    NOTE: you may need to alter the magic_file for your system to point to the magic file."""
    # if 'magic' in sys.modules:
    #     try:
    #         ms = magic.open(magic.MAGIC_NONE)
    #         ms.load()
    #         return ms.buffer(data)
    #     except:
    #         try:
    #             return magic.from_buffer(data)
    #         except magic.MagicException:
    #             magic_custom = magic.Magic(magic_file='C:\windows\system32\magic')
    #             return magic_custom.from_buffer(data)
    return ''

def get_ssdeep(filename):
    """There are two Python bindings for ssdeep, each with a different interface. So we try
    Jose's pyssdeep first and if it fails, try the one from pypi. Just install one or the other:
    http://code.google.com/p/pyssdeep/
    http://pypi.python.org/packages/source/s/ssdeep/ssdeep-2.5.tar.gz#md5=fd9e5271c01ca389cc621ae306327ab6
    """
    try:
        from ssdeep import ssdeep
        s = ssdeep()
        return s.hash_file(filename)
    except:
        try:
            import ssdeep
            return ssdeep.hash_from_file(filename)
        except:
            pass
    return ''

class PEScanner:
    def __init__(self, files, yara_rules=None, peid_sigs=None):
        self.files = files

        # initialize YARA rules if provided
        if yara_rules and 'yara' in sys.modules:
            self.rules = yara.compile(yara_rules)
        else:
            self.rules = None

        # initialize PEiD signatures if provided
        if peid_sigs:
            self.sigs = peutils.SignatureDatabase(peid_sigs)
        else:
            self.sigs = None

    def check_ep_section(self, pe):
        """ Determine if a PE's entry point is suspicious """
        name = ''
        ep = pe.OPTIONAL_HEADER.AddressOfEntryPoint
        pos = 0
        for sec in pe.sections:
            if (ep >= sec.VirtualAddress) and \
               (ep < (sec.VirtualAddress + sec.Misc_VirtualSize)):
                # name = sec.Name.replace('\x00', '')
                name = sec.Name
                break
            else:
                pos += 1
        return (ep, name, pos)

    def check_verinfo(self, pe):
        """ Determine the version info in a PE file """
        ret = []

        # if hasattr(pe, 'VS_VERSIONINFO'):
        #     if hasattr(pe, 'FileInfo'):
        #         for entry in pe.FileInfo:
        #             if hasattr(entry, 'StringTable'):
        #                 for st_entry in entry.StringTable:
        #                     for str_entry in st_entry.entries.items():
        #                         ret.append(convert_to_printable(str_entry[0]) + ': ' + convert_to_printable(str_entry[1]) )
        #             elif hasattr(entry, 'Var'):
        #                 for var_entry in entry.Var:
        #                     if hasattr(var_entry, 'entry'):
        #                         ret.append(convert_to_printable(var_entry.entry.keys()[0]) + ': ' + var_entry.entry.values()[0])
        if hasattr(pe, 'FileInfo'):
          for fileinfo in pe.FileInfo[0]:
            if fileinfo.Key.decode() == 'StringFileInfo':
              for st in fileinfo.StringTable:
                for entry in st.entries.items():
                  ret.append(entry[0].decode() + ": " + entry[1].decode())

        return '\n'.join(ret)

    def check_tls(self, pe):
        callbacks = []
        if (hasattr(pe, 'DIRECTORY_ENTRY_TLS') and \
                    pe.DIRECTORY_ENTRY_TLS and \
                    pe.DIRECTORY_ENTRY_TLS.struct and \
                    pe.DIRECTORY_ENTRY_TLS.struct.AddressOfCallBacks):
            callback_array_rva = pe.DIRECTORY_ENTRY_TLS.struct.AddressOfCallBacks - pe.OPTIONAL_HEADER.ImageBase
            idx = 0
            while True:
                func = pe.get_dword_from_data(pe.get_data(callback_array_rva + 4 * idx, 4), 0)
                if func == 0:
                    break
                callbacks.append(func)
                idx += 1
        return callbacks

    def check_rsrc(self, pe):
        ret = {}
        if hasattr(pe, 'DIRECTORY_ENTRY_RESOURCE'):
            i = 0
            for resource_type in pe.DIRECTORY_ENTRY_RESOURCE.entries:
                if resource_type.name is not None:
                    name = "%s" % resource_type.name
                else:
                    name = "%s" % pefile.RESOURCE_TYPE.get(resource_type.struct.Id)
                if name == None:
                    name = "%d" % resource_type.struct.Id
                if hasattr(resource_type, 'directory'):
                    for resource_id in resource_type.directory.entries:
                        if hasattr(resource_id, 'directory'):
                            for resource_lang in resource_id.directory.entries:
                                data = pe.get_data(resource_lang.data.struct.OffsetToData, resource_lang.data.struct.Size)
                                filetype = get_filetype(data)
                                lang = pefile.LANG.get(resource_lang.data.lang, '*unknown*')
                                sublang = pefile.get_sublang_name_for_lang( resource_lang.data.lang, resource_lang.data.sublang )
                                ret[i] = (name, resource_lang.data.struct.OffsetToData, resource_lang.data.struct.Size, filetype, lang, sublang)
                                i += 1
        return ret

    def check_imports(self, pe):
        ret = []
        if not hasattr(pe, 'DIRECTORY_ENTRY_IMPORT'):
            return ret
        for lib in pe.DIRECTORY_ENTRY_IMPORT:
            for imp in lib.imports:
                if (imp.name != None) and (imp.name != ""):
                    for alert in alerts:
                        imppname = "".join([chr(c) for c in imp.name if chr(c) in string.printable])
                        if imppname.startswith(alert):
                            ret.append(imppname)
        return ret

    def get_timestamp(self, pe):
        val = pe.FILE_HEADER.TimeDateStamp
        ts = '0x%-8X' % (val)
        try:
            ts += ' [%s UTC]' % time.asctime(time.gmtime(val))
            that_year = time.gmtime(val)[0]
            this_year = time.gmtime(time.time())[0]
            if that_year < 2000 or that_year > this_year:
                ts += " [SUSPICIOUS]"
        except:
            ts += ' [SUSPICIOUS]'
        return ts

    def check_packers(self, pe):
        packers = []
        if self.sigs:
            matches = self.sigs.match(pe, ep_only = True)
            if matches != None:
                for match in matches:
                    packers.append(match)
        return packers

    def check_yara(self, data):
        ret = []
        if self.rules:
            yarahits = self.rules.match(data=data)
            if yarahits:
              for hit in yarahits:
                ret.append("YARA: %s" % hit.rule)
                #for key, val in hit.strings.iteritems():
                for (key,stringname,val) in hit.strings:
                    makehex = False
                    for char in val:
                        if char not in string.printable:
                            makehex = True
                            break
                    if makehex == True:
                        ret.append("   %s => %s" % (hex(key), binascii.hexlify(val)))
                    else:
                        ret.append("   %s => %s" % (hex(key), val))
        return '\n'.join(ret)

    def check_clam(self, file):
        if os.path.isfile(clamscan_path):
            status, output = commands.getstatusoutput("%s %s" % (clamscan_path, file))
            if status != 0:
                return "Clamav: %s" % output.split("\n")[0]
        return ''

    def header(self, msg):
        return "\n" + msg + "\n" + ("=" * 80)

    def collect(self):
        count = 0

        for file in self.files:
            out = []

            try:
                FILE = open(file, "rb")
                data = FILE.read()
                FILE.close()
            except:
                # continue
                raise('File cannot be open!!')
                return None

            if data == None or len(data) == 0:
                out.append("Cannot read %s (maybe empty?)" % file)
                out.append("")
                continue

            try:
                pe = pefile.PE(data=data, fast_load=True)
                pe.parse_data_directories( directories=[
                    pefile.DIRECTORY_ENTRY['IMAGE_DIRECTORY_ENTRY_IMPORT'],
                    pefile.DIRECTORY_ENTRY['IMAGE_DIRECTORY_ENTRY_EXPORT'],
                    pefile.DIRECTORY_ENTRY['IMAGE_DIRECTORY_ENTRY_TLS'],
                    pefile.DIRECTORY_ENTRY['IMAGE_DIRECTORY_ENTRY_RESOURCE']])
            except:
                out.append("Cannot parse %s (maybe not PE?)" % file)
                out.append("")
                #continue

            out.append(("#" * 80) + "\nRecord %d\n" % count + ("#" * 80))
            out.append(self.header("Meta-data"))
            out.append("File:    %s" % file)
            out.append("Size:    %d bytes" % len(data))
            out.append("Type:    %s" % get_filetype(data))
            out.append("MD5:     %s"  % hashlib.md5(data).hexdigest())
            out.append("SHA1:    %s" % hashlib.sha1(data).hexdigest())
            out.append("ssdeep:  %s" % get_ssdeep(file))
            out.append("Date:    %s" % self.get_timestamp(pe))

            # Alert if the EP section is not in a known good section or if its in the last PE section
            (ep, name, pos) = self.check_ep_section(pe)
            name = "".join([chr(c) for c in name if chr(c) in string.printable])
            s = "EP:      %s %s %d/%d" % (hex(ep+pe.OPTIONAL_HEADER.ImageBase), name, pos, len(pe.sections))
            if (name not in good_ep_sections) or pos == len(pe.sections):
                s += " [SUSPICIOUS]"
            out.append(s)

            crc_claimed = pe.OPTIONAL_HEADER.CheckSum
            crc_actual  = pe.generate_checksum()
            out.append("CRC:     Claimed: 0x%x, Actual: 0x%x %s" % (
                crc_claimed, crc_actual, "[SUSPICIOUS]" if crc_actual != crc_claimed else ""))

            packers = self.check_packers(pe)
            if len(packers):
                out.append("Packers: %s" % ','.join(packers))

            if 'yara' in sys.modules:
                yarahits = self.check_yara(data)
            else:
                yarahits = []

            clamhits = self.check_clam(file)

            if len(yarahits) or len(clamhits):
                out.append(self.header("Signature scans"))
                out.append(yarahits)
                out.append(clamhits)

            callbacks = self.check_tls(pe)
            if len(callbacks):
                out.append(self.header("TLS callbacks"))
                for cb in callbacks:
                    out.append("    0x%x" % cb)

            resources = self.check_rsrc(pe)
            if len(resources):
                out.append(self.header("Resource entries"))
                out.append("%-18s %-8s %-8s %-12s %-24s %s" % ("Name", "RVA", "Size", "Lang", "Sublang", "Type"))
                out.append("-" * 80)
                for rsrc in resources.keys():
                    (name,rva,size,rtype,lang,sublang) = resources[rsrc]
                    out.append("%-18s %-8s %-8s %-12s %-24s %s" % (name, hex(rva), hex(size), lang, sublang, rtype))

            # source: https://code.google.com/p/pyew/
            if hasattr(pe, 'DIRECTORY_ENTRY_IMPORT'):
              imports_total = len(pe.DIRECTORY_ENTRY_IMPORT)
              if imports_total > 0:
                  c = 1
                  out.append(header("Imports"))
                  for entry in pe.DIRECTORY_ENTRY_IMPORT:
                      out.append("[%s] %s" % (c, entry.dll.decode()))
                      if verb == True:
                          for imp in entry.imports:
                              if (imp.name != None) and (imp.name != ""):
                                  out.append("\t%s %s" % (hex(imp.address),imp.name))
                      c += 1

            imports = self.check_imports(pe)
            if len(imports):
                out.append(self.header("Suspicious IAT alerts"))
                for imp in imports:
                    out.append(imp)

            out.append(self.header("Sections"))
            out.append("%-10s %-12s %-12s %-12s %-12s" % ("Name", "VirtAddr", "VirtSize", "RawSize", "Entropy"))
            out.append("-" * 80)

            for sec in pe.sections:
                s = "%-10s %-12s %-12s %-12s %-12f" % (
                    ''.join([chr(c) for c in sec.Name if chr(c) in string.printable]),
                    hex(sec.VirtualAddress),
                    hex(sec.Misc_VirtualSize),
                    hex(sec.SizeOfRawData),
                    sec.get_entropy())
                if sec.SizeOfRawData == 0 or (sec.get_entropy() > 0 and sec.get_entropy() < 1) or sec.get_entropy() > 7:
                    s += "[SUSPICIOUS]"
                out.append(s)

            verinfo = self.check_verinfo(pe)
            if len(verinfo):
                out.append(self.header("Version info"))
                out.append(verinfo)

            out.append("")
            # print ('\n'.join(out))
            return '\n'.join(out)
            count += 1
verb = False

def get_infos(files = []):
    pescan = PEScanner(files, '', '')
    return pescan.collect()

python-magic is not installed, file types will not be available


In [ ]:
def isdel(st):
    if len(st) < 2:
        return True
    if st.isspace():
        return True
    if st.startswith('=='):
        return True
    if st.startswith('--'):
        return True
    return False

In [ ]:
def get_attr(data):
    
    data = data.split('\n')
    iat = []
    imps = []
    sec_names = []
    rentries = []
    for inx, line in enumerate(data):

        if line.startswith('Suspicious IAT alerts'):
            i = inx+2
            while i < len(data) and isdel(data[i]) == False:
                d = re.sub('\[[0-9]+\]', '', data[i]).strip()
                iat.append(d)
                i += 1

        elif line.startswith('Imports'):
            i = inx+2
            while i < len(data) and isdel(data[i]) == False:
                d = re.sub('\[[0-9]+\]', '', data[i]).strip()
                imps.append(d)
                i += 1

        elif line.startswith('Resource entries'):
            i = inx+4
            while i < len(data) and isdel(data[i]) == False:
                rentries.append(re.split('[ :]+', data[i])[0].strip())
                i += 1

        elif line.startswith('Sections'):
            i = inx+4
            while i < len(data) and isdel(data[i]) == False:
                info = re.split(' +', data[i])
                sec_names.append(info[0].strip())
                i += 1
        if len(iat) == 0:
          iat.append('no_iat')
        if len(imps) == 0:
          imps.append('no_imports')
    return " ".join(iat) +" " + " ".join(rentries)+ " " + " ".join(imps) + " " + " ".join(sec_names)

def create_soup(text):
    l = list(set(word_tokenize(text)))
    return " ".join(l)

===============================================================

In [ ]:
infos = get_infos(['/content/IBALauncher.exe'])

In [ ]:
print(infos)

################################################################################
Record 0
################################################################################

Meta-data
File:    /content/IBALauncher.exe
Size:    40558312 bytes
Type:    
MD5:     dda07e6c6e618d842f8d8990fb72ebd5
SHA1:    5fe186c2fce27d62bbf8ca23794d182ec51247a5
ssdeep:  
Date:    0x5E11D3A3 [Sun Jan  5 12:16:35 2020 UTC]
EP:      0x407cd3 .text 0/6
CRC:     Claimed: 0x26bbb6a, Actual: 0x26bbb6a 

Resource entries
Name               RVA      Size     Lang         Sublang                  Type
--------------------------------------------------------------------------------
RT_ICON            0x3c238  0x468    LANG_NEUTRAL SUBLANG_NEUTRAL          
RT_ICON            0x3c6a0  0x10a8   LANG_NEUTRAL SUBLANG_NEUTRAL          
RT_ICON            0x3d748  0x25a8   LANG_NEUTRAL SUBLANG_NEUTRAL          
RT_ICON            0x3fcf0  0x4228   LANG_NEUTRAL SUBLANG_NEUTRAL          
RT_ICON            0x43f18  0x25a8   L

In [ ]:
features = create_soup(get_attr(infos))
features

'USER32.dll CreateProcessW WS2_32.dll KERNEL32.dll RT_GROUP_ICON no_iat .gfids .rsrc .reloc RT_MANIFEST .rdata ADVAPI32.dll .text .data no_imports RT_ICON'

In [ ]:
with open('/content/models.pkl', 'rb') as fp1:
    out_models = pickle.load(fp1)
    with open('/content/tfidff.pkl', 'rb') as fp2:
        tfidff = pickle.load(fp2)
        inp = tfidff.transform([features])
        result = 'Malware' if out_models[0].predict(inp)[0] == 1 else 'Benigns'
        print(result)

NameError: ignored

**Testing for new Malwares**

In [ ]:
# infos = get_infos(['/content/malwares/malwares/winrar-64Bit-400.exe'])
# features = create_soup(get_attr(infos))
# print(features)

In [ ]:
# with open('/content/models.pkl', 'rb') as fp1:
#     out_models = pickle.load(fp1)
#     with open('/content/tfidff.pkl', 'rb') as fp2:
#         tfidff = pickle.load(fp2)
#         inp = tfidff.transform(['.no_imports'])
#         result = 'Malware' if out_models[0].predict(inp)[0] == 1 else 'Benigns'
#         print(result)
        # inp = tfidff.transform([' text RT_ACCELERATOR data'])
        # display(inp.toarray())
        
        # print(tfidff.get_feature_names_out())

In [ ]:
LSTMmodel = load_model('LSTMmodel.h5')

In [ ]:
with open('/content/models.pkl', 'rb') as fp1:
  out_models = pickle.load(fp1)
  with open('/content/tfidff.pkl', 'rb') as fp2:
      tfidff = pickle.load(fp2)
      with open('/content/tok.pkl', 'rb') as fp2:
        tok = pickle.load(fp2)

      for filename in os.listdir(r'/content/malwares'):
        infos = get_infos([f'/content/malwares/{filename}'])
        features = create_soup(get_attr(infos))
        # print(features)
        inp = tfidff.transform([features])
        print('DecisionTreeClassifier: ')
        result = 'Malware' if out_models[0].predict(inp)[0] == 1 else 'Benigns'
        print(f'{filename} => {result}')

        print('LogisticRegression: ')
        result = 'Malware' if out_models[1].predict(inp)[0] == 1 else 'Benigns'
        print(f'{filename} => {result}')

        print('KNeighborsClassifier: ')
        result = 'Malware' if out_models[2].predict(inp)[0] == 1 else 'Benigns'
        print(f'{filename} => {result}')

        print('===============================================')
        test_sequences = tok.texts_to_sequences([features])
        test_sequences_matrix = pad_sequences(test_sequences, maxlen=150)
        result = LSTMmodel.predict(test_sequences_matrix, verbose=0)
        print('Probability of being a Malware is {}'.format(round(result[0][0]*100, 2)))
        print('===============================================\n')
        # print(inp.shape)
        # inp = tfidff.transform([features])
        # display(inp.toarray())

DecisionTreeClassifier: 
mcpatcher.exe => Malware
LogisticRegression: 
mcpatcher.exe => Benigns
KNeighborsClassifier: 
mcpatcher.exe => Malware
Probability of being a Malware is 99.99

DecisionTreeClassifier: 
854137.exe => Malware
LogisticRegression: 
854137.exe => Benigns
KNeighborsClassifier: 
854137.exe => Malware
Probability of being a Malware is 99.98

DecisionTreeClassifier: 
DropboxInstaller.exe => Malware
LogisticRegression: 
DropboxInstaller.exe => Benigns
KNeighborsClassifier: 
DropboxInstaller.exe => Malware
Probability of being a Malware is 99.99

DecisionTreeClassifier: 
Bombermania.exe => Malware
LogisticRegression: 
Bombermania.exe => Malware
KNeighborsClassifier: 
Bombermania.exe => Malware
Probability of being a Malware is 100.0

DecisionTreeClassifier: 
newbos2.exe => Malware
LogisticRegression: 
newbos2.exe => Benigns
KNeighborsClassifier: 
newbos2.exe => Malware
Probability of being a Malware is 99.14

DecisionTreeClassifier: 
2d.exe => Malware
LogisticRegression: 

In [ ]:
# import shutil
# shutil.make_archive('/content/malwares','zip','/content/malwares')

In [ ]:
# !unzip malwares.zip